In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt

import numpy as np
import pandas as pd

import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Utilities as uu

import Models.Soybean_USA_Yield_GA as sy

import APIs.Bloomberg as ba

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [ ]:
scope = sy.Define_Scope()
raw_data = sy.Get_Data_All_Parallel(scope)    
raw_data['w_w_df_all']['hist']

In [3]:
m=um.pick_model('GA_soy',72)
ww = um.from_cols_to_var_windows(m.params.index)

Start Deserializing: GA_soy
Done Deserializing: GA_soy


In [9]:
model_df = um.extract_yearly_ww_variables(w_df = raw_data['w_w_df_all']['hist'],var_windows= ww)

In [11]:
model_df
raw_data.keys()

dict_keys(['years', 'locations', 'yield', 'weights', 'planting_progress', 'blooming_progress', 'w_df_all', 'w_w_df_all'])

In [15]:
model_df['year']=model_df.index
model_df

,USA_TempMax_Aug21-Aug28,USA_Sdd30_Jun17-Aug19,USA_TempMax_Aug16-Aug19,USA_Sdd30_Jul04-Aug20,USA_Sdd30_Jun27-Jul15,year
year,,,,,,
1985,24.624972,0.535132,26.473488,0.628962,1.012893,1985
1986,26.659765,0.921108,29.105613,0.923356,0.865736,1986
1987,24.709041,1.342054,30.206546,1.565715,0.324604,1987
1988,27.760942,2.973521,35.356721,3.018119,2.246986,1988
1989,28.301939,0.903515,26.984517,0.970026,1.707921,1989
1990,28.775181,0.622473,29.335568,0.545376,1.423345,1990
1991,31.050750,1.110971,28.869216,1.043178,1.501142,1991
1992,25.869936,0.177898,24.470790,0.162438,0.225634,1992
1993,30.049589,0.446428,30.398349,0.514464,0.458910,1993


In [5]:
m.params

const                     -960.550291
year                         0.509614
USA_TempMax_Aug21-Aug28     -0.383391
USA_Sdd30_Jun17-Aug19        1.804934
USA_TempMax_Aug16-Aug19     -0.348013
USA_Sdd30_Jul04-Aug20       -0.909144
USA_Sdd30_Jun27-Jul15        0.800158
dtype: float64

In [ ]:
f='GA_soy'
dm_best = {}
dm_best[f] = uu.deserialize(f)
r=dm_best[f]

r.keys()

In [ ]:
print(len(r['model']))
print(len(r['corr']))
print(r['cv_p'][0])